In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F
import pandas as pd
import re
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packa

In [2]:
# def pool(hidden_state, mask, pooling_method="csl"): # переиспользовать
#     if pooling_method == "mean":
#         s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
#         d = mask.sum(axis=1, keepdim=True).float()
#         return s / d
#     elif pooling_method == "cls":
#         return hidden_state[:, 0]


# def prepare_input(text): # переиспользовать
#     inputs = tokenizer.encode_plus(
#         text,
#         return_tensors=None,
#         add_special_tokens=True,
#         max_length=512,
#         pad_to_max_length=True,
#         truncation=True,
#     )
#     for k, v in inputs.items():
#         inputs[k] = torch.tensor([v], dtype=torch.long)
#     return inputs

# def get_vector(text): # переиспользовать
#     inputs = prepare_input(text)
#     outputs = model(**inputs)
#     embeddings = pool(
#             outputs.last_hidden_state,
#             inputs["attention_mask"],
#             pooling_method="cls" # or try "mean"
#         )
#     return F.normalize(embeddings, p=2, dim=1).tolist()[0]

# def nodes_from_tables():
#     nodes = []
#     df = pd.read_excel("../case/01_База_знаний.xlsx").sample(n=10)
#     df.drop(columns=["Классификатор 1 уровня", "Классификатор 2 уровня", "Тема"], inplace=True)

#     for i, row in df.iterrows():
#         node = TextNode(text=row["Ответ из БЗ"], embedding=get_vector(row["Вопрос из БЗ"]))
#         nodes.append(node)

#     df = pd.read_excel("../case/02_Реальные_кейсы.xlsx").sample(n=10)
#     df.drop(columns=["Вопрос из БЗ", "Ответ из БЗ", "Классификатор 1 уровня", "Классификатор 2 уровня"], inplace=True)

#     for i, row in df.iterrows():
#         node = TextNode(text=row["Ответ сотрудника"], embedding=get_vector(row["Вопрос пользователя"]))
#         nodes.append(node)

#     return nodes

In [3]:
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=128)

docs = SimpleDirectoryReader(input_files=[
                             "../case/03_ГЕНЕРАЛЬНОЕ ПОЛЬЗОВАТЕЛЬСКОЕ СОГЛАШЕНИЕ RUTUBE.docx", "../case/04_УСЛОВИЯ РАЗМЕЩЕНИЯ КОНТЕНТА.docx"]).load_data()
nodes = node_parser.get_nodes_from_documents(docs)

for node in nodes:
    node.text = re.sub("\\n", " ", node.text)
    node.text = re.sub(" +", " ", node.text)

embed_model = HuggingFaceEmbedding(
    model_name="../api/encoder_model"
)
# model_name = "ai-forever/ru-en-RoSBERTa"
# model = AutoModel.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

model_llm = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer_llm = AutoTokenizer.from_pretrained(MODEL_NAME)

llm = HuggingFaceLLM(
    model=model_llm,
    tokenizer=tokenizer_llm,
    max_new_tokens=256,
    generate_kwargs={
        "temperature": 0.00001,
        "do_sample": False,
        "repetition_penalty": 1.12,
        # "top_p": 0.9,
        # "top_k": 30,
        "pad_token_id": 128000,
        "eos_token_id": 128009,
        "bos_token_id": 128000},
    system_prompt="Ты умный ассистент которого зовут Наташа. Ты любишь отвечать на вопросы пользователей.",
    device_map="auto",
)

index = VectorStoreIndex(nodes, embed_model=embed_model)
# table_nodes = nodes_from_tables()
# index.insert_nodes(table_nodes)

vector_query_engine = index.as_retriever(similarity_top_k=10)

No sentence-transformers model found with name ../api/encoder_model. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
template = PromptTemplate(
    "Ниже представлена информация из правовых документов:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "На основе вышеуказанной информации из документов, ответь на вопрос пользователя. В ответе укажи только содержательную часть без повторения вопроса.\n"
    "Вопрос: {query}\n"
    "Ответ:"
)

query = "Где индексируется размещенный контент?"
# query = "Как тебя зовут?"
nodes = vector_query_engine.retrieve(query)
context_str = "\n\n".join([n.node.get_content() for n in nodes])
response = llm.complete(
    template.format(context_str=context_str, query=query)
)
response_clear = re.findall(
    r"[а-яА-Я\n \.\,\(\)a-zA-Z\!\?\/\:0-9\"\'\;\-\—\-]+(?=---)", response.text)[0]
response_clear = response_clear.strip()

/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/home/pc/Desktop/cp-r

In [5]:
print(response_clear)

Индексирование размещенного контента происходит в глобальных поисковых системах, таких как Google, Bing, Yandex и другие, в результате чего результаты индексирования могут отображаться в поисковой выдаче этих систем в виде ссылок на контентное содержимое канала пользователя. Это происходит автоматически, без участия администрации RUTUBE, и является естественным процессом работы глобальной сети Интернет. Браузеры, такие как Internet Explorer, Mozilla, Google Chrome, Opera и др., кэшируют контент, который был ранее просмотрен пользователем, что позволяет быстро открывать уже просматриваемые страницы. Однако это не связано с незаконным использованием объектов интеллектуальной собственности. Кэширование производится браузером в автоматическом режиме в соответствии с определенными алгоритмами.
